# Calculate optimal routes for six trucks with multiple waypoints


**Table of contents**:
* [Introduction](#Introduction)
* [Calculate optimized truck route (returning to the starting point)](#Calculate-optimized-truck-route-(returning-to-the-starting-point))
* [Calculate all routes](#Calculate-all-routes)
* [Display calculated routes on a Map](#Display-calculated-routes-on-a-Map)

## Introduction

When we have a fleet of trucks that have to distribute stock to several shops, we will face the **challenge of calculating the optimal route for each truck**.  
![Animation of truck optimized routes](./images/truck_optimized_routes.gif)

In [ ]:
!pip3 install boto3
!pip3 install leafmap

In [ ]:
import boto3
from dotenv import load_dotenv
import os
import pandas as pd

In [ ]:
# Load environment variables from .env file
load_dotenv()

# Create a client for Amazon Location service
amazon_location_client = boto3.client(
    "location",
    aws_access_key_id = os.getenv("AWS_ACCESS_KEY"),
    aws_secret_access_key = os.getenv("AWS_SECRET_ACCESS_KEY"),
    region_name = os.getenv("AWS_REGION"),
)

## Calculate optimized truck route (returning to the starting point)

In [ ]:
response = amazon_location_client.calculate_route(
    CalculatorName='AWS-Esri-Demo',
    #DepartNow=True|False,
    DeparturePosition = [-0.1417089267, 51.5787616672],
    #DepartureTime = datetime(2015, 1, 1),
    DestinationPosition = [-0.1417089267, 51.5787616672],
    DistanceUnit = 'Kilometers',
    IncludeLegGeometry = True,
    #Key='string',
    TravelMode = 'Truck',
    TruckModeOptions = {
        'AvoidFerries': True,
        'AvoidTolls': True,
        'Dimensions': {
            'Height': 2,
            'Length': 2.57,
            'Unit': 'Meters',
            'Width': 1.4
        }
    },
    WaypointPositions = [
        [-0.1254974319, 51.50852485], 
        [-0.08229997944, 51.51856002], 
        [-0.1259365099, 51.53253756], 
        [-0.1425202953, 51.49654998]
    ]
)

In [ ]:
len(response['Legs'])

In [ ]:
response['Legs'][0]

In [ ]:
response['Summary']

# Calculate all routes

In [ ]:
df = pd.read_csv("truck_data.csv")

df

In [ ]:
from ast import literal_eval
routes = []
for index, fila in df.iterrows():
    routes.append(amazon_location_client.calculate_route(
        CalculatorName='AWS-Esri-Demo',
        DeparturePosition = literal_eval(df.at[index,'origin']),
        DestinationPosition = literal_eval(df.at[index,'origin']),
        DistanceUnit = 'Kilometers',
        IncludeLegGeometry = True,
        TravelMode = 'Truck',
        TruckModeOptions = {
            'AvoidFerries': True,
            'AvoidTolls': True,
            'Dimensions': {
                'Height': df.at[index,'height'],
                'Length': df.at[index,'length'],
                'Unit': 'Meters',
                'Width': df.at[index,'width']
            }
        },
        WaypointPositions = literal_eval(df.at[index,'destinations'])
    ))


In [ ]:
routes

# Display calculated routes on a Map

In [ ]:
import geojson
# Create a GeoJSON FeatureCollection
line_strings = []

def convert_to_geojson(line_strings):
    # Create a GeoJSON FeatureCollection
    features = []

    # Add LineString features
    for line_string_coords in line_strings:
        line_string = geojson.LineString(coordinates=line_string_coords)
        features.append(geojson.Feature(geometry=line_string))

    # Create the FeatureCollection
    feature_collection = geojson.FeatureCollection(features)

    # Serialize the GeoJSON object to a string
    geojson_str = geojson.dumps(feature_collection, sort_keys=True)

    return geojson_str

for i, route in enumerate(routes):
    line_strings = []
    for leg in route['Legs']:
        line_strings.append(leg['Geometry']['LineString'])

    # Call the function to convert to GeoJSON
    result_geojson = convert_to_geojson(line_strings)

    file_path = "data/truck/route_truck%d.geojson" % (i)

    # Write the GeoJSON string to the file
    with open(file_path, "w") as geojson_file:
        geojson_file.write(result_geojson)

    print(f"GeoJSON saved to {file_path}")

In [ ]:
import leafmap

m = leafmap.Map(center=[51.50852507962595, -0.12549702207699787], zoom=12, draw_control=False, measure_control=False)
m.add_basemap("Esri.WorldTopoMap")
#m.add_xy_data('uk_stores_with_coordinates.csv', x="longitude", y="latitude", layer_name="All stores")
for i in range(6):
    file_path = "truck%d_origin.geojson" % (i)
    m.add_geojson("data/truck/%s" % (file_path), layer_name=file_path)
    file_path = "truck%d_destinations.geojson" % (i)
    m.add_geojson("data/truck/%s" % (file_path), layer_name=file_path)
    file_path = "route_truck%d.geojson" % (i)
    m.add_geojson("data/truck/%s" % (file_path), layer_name="truck%d_route" % (i))

m